<a href="https://colab.research.google.com/github/JaeKyunKo/Darknet_YOLOv3/blob/master/YOLOv3_tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Verify CUDA Version
!/usr/local/cuda/bin/nvcc --version 

In [0]:
# Get Darknet Platform
!git clone https://github.com/JinFree/darknet

In [0]:
# Make Darknet Preparation
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [0]:
# Make darknet (build)
!make

In [0]:
# Define Helper Functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [0]:
# Get Access to Google Drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# Check Files in Google Drive
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [0]:
# Check Files
!ls /mydrive/images/

In [0]:
!cp /mydrive/yolov3/obj.zip /content/darknet/

In [0]:
!unzip /content/darknet/obj.zip -d /content/darknet/data/

In [0]:
!cp /mydrive/yolov3_tiny/yolov3_tiny_custom.cfg /content/darknet/cfg/

In [0]:
!cp /mydrive/yolov3_tiny/obj.names /content/darknet/data/
!cp /mydrive/yolov3_tiny/obj.data /content/darknet/data/

In [0]:
!cp /mydrive/yolov3/generate_train.py /content/darknet

In [0]:
!python /content/darknet/generate_train.py

In [0]:
# Upload Pretrained Convolutional Layer Weights from GDrive
!cp /mydrive/yolov3_tiny/yolov3-tiny.conv.15 /content/darknet/

In [0]:
#Train from Scratch
%cd /content/darknet/
!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolov3_tiny_custom.cfg /content/darknet/yolov3-tiny.conv.15 -dont_show

In [0]:
# Train From Saved File
%cd /content/darknet/
!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolov3_tiny_custom.cfg /mydrive/yolov3_tiny/backup/yolov3_tiny_custom_last.weights -dont_show

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
v3 (mse loss, Normalizer: (iou: 0.750000, cls: 1.000000) Region 16 Avg (IOU: 0.827197, GIOU: 0.818313), Class: 0.999953, Obj: 0.993131, No Obj: 0.011050, .5R: 1.000000, .75R: 1.000000, count: 2
v3 (mse loss, Normalizer: (iou: 0.750000, cls: 1.000000) Region 23 Avg (IOU: -nan, GIOU: -nan), Class: -nan, Obj: -nan, No Obj: 0.000000, .5R: -nan, .75R: -nan, count: 0
v3 (mse loss, Normalizer: (iou: 0.750000, cls: 1.000000) Region 16 Avg (IOU: 0.857054, GIOU: 0.851252), Class: 0.999792, Obj: 0.988061, No Obj: 0.003048, .5R: 1.000000, .75R: 1.000000, count: 1
v3 (mse loss, Normalizer: (iou: 0.750000, cls: 1.000000) Region 23 Avg (IOU: 0.702535, GIOU: 0.692434), Class: 1.000000, Obj: 0.998148, No Obj: 0.000754, .5R: 1.000000, .75R: 0.000000, count: 1
v3 (mse loss, Normalizer: (iou: 0.750000, cls: 1.000000) Region 16 Avg (IOU: 0.873205, GIOU: 0.871487), Class: 0.996515, Obj: 0.984280, No Obj: 0.003556, .5R: 1.000000, .75R: 1.000000, count: 1
v3 (mse loss, Norm

In [0]:
# Parameter Reduction
%cd /content/darknet/cfg
!sed -i 's/batch=64/batch=1/' yolov3_tiny_custom.cfg
!sed -i 's/subdivisions=64/subdivisions=1/' yolov3_tiny_custom.cfg
%cd ..

In [0]:
# Image Validation
%cd /content/darknet
!./darknet detector test /content/darknet/data/obj.data /content/darknet/cfg/yolov3_tiny_custom.cfg /mydrive/yolov3/backup/yolov3_custom_last.weights /mydrive/images/test4.jpg
imShow('predictions.jpg')

In [0]:
# Video Validation
%cd /content/darknet
!./darknet detector demo /content/darknet/data/obj.data /content/darknet/cfg/yolov3_custom.cfg /mydrive/yolov3/backup/yolov3_custom_last.weights -dont_show /mydrive/videos/mask.mp4 -i 0 -out_filename yolov3_out.avi